In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


# LCEL Interface

- `invoke` : 입력에 대한 체인 호출. LLM 답변 생성 완료 후 확인 가능  
- `stream` : 응답 청크 스트리밍 
- `batch` : 입력 목록에 대한 체인 호출

(참고) 비동기
- `ainvoke` : 비동기적으로 입력에 대해 체인 호출
- `astream` : 비동기적으로 응답 청크 스트리밍
- `abatch` : 비동기적으로 입력 목록에 대해 체인 호출 
- `astream_log` : 최종 응답 뿐만 아니라 발생 중간 단계를 스트리밍

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response

In [5]:
# template 정의
template = "{topic}에 대하여 3문장으로 설명해 주세요."

# PromptTemplate 생성
prompt = PromptTemplate.from_template(template)

# model
model = ChatOpenAI()

# Output Parser
output_parser = StrOutputParser()

# Chian
chain = prompt | model | output_parser

# stream

In [6]:
# stream, 실시간 출력
# end="" : 출력 후 줄바꿈 하지 않음 
# flush=True : 출력 버터를 즉시 비움 

for token in chain.stream({"topic":"멀티모달"}):
    print(token, end="", flush=True)

멀티모달은 여러 가지 형태의 통신 수단을 조합하여 정보를 전달하거나 상호작용하는 방식을 말합니다. 이는 텍스트, 음성, 이미지, 비디오 등 다양한 매체를 활용하여 사용자 경험을 향상시키는 것을 목표로 합니다. 인공지능 기술의 발전으로 멀티모달 인터페이스가 더욱 발전하고 있어, 사용자와의 상호작용이 더욱 손쉽고 자연스러워지고 있습니다.

# invoke 

In [8]:
chain.invoke({"topic":"스타벅스"})

'스타벅스는 세계적으로 유명한 커피 전문 브랜드이며, 다양한 음료와 디저트를 판매하고 있다. 매장 내부는 트렌디하고 아늑한 분위기로 인기가 많으며, 매일 수많은 고객들이 방문한다. 전 세계에 많은 지점을 두고 있어 어디서든 쉽게 만날 수 있는 커피 브랜드이다.'

# batch 

In [9]:
chain.batch(
    [
        {"topic":"스타벅스"},
        {"topic":"Instagram"},
    ]
)

['스타벅스는 세계적으로 유명한 커피 전문 브랜드로, 다양한 음료와 디저트를 판매합니다. 매장 분위기는 아늑하고 편안하며, 고객들이 휴식을 취하거나 모임을 즐기기에 좋은 장소입니다. 또한 매장마다 다양한 음료 메뉴와 특별한 이벤트를 제공하여 소비자들에게 다양한 즐길거리를 제공합니다.',
 '인스타그램은 사진과 영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 다양한 사진과 동영상을 통해 공유할 수 있습니다. 또한 팔로워를 통해 다른 사람들의 활동을 구독하고 소통할 수 있으며, 해시태그를 통해 관심사에 맞는 콘텐츠를 찾을 수 있습니다. 인스타그램은 시각적인 콘텐츠를 중심으로 한 SNS로, 다양한 필터와 편집 기능을 통해 사용자들이 자신만의 콘텐츠를 만들고 공유할 수 있습니다.']

`max_concurrency`: 매개변수를 사용하여 동시 요청 개수 설정

In [10]:
chain.batch(
    [
        {"topic":"스타벅스"},
        {"topic":"Instagram"},
        {"topic":"chatGPT"},
        {"topic":"한글"},
        {"topic":"머신러닝"},
        {"topic":"뉴런"},
    ],
    config={"max_concurrency":3},
)

['스타벅스는 전 세계적으로 유명한 커피 전문 브랜드이며, 다양한 음료와 디저트를 판매합니다. 고객들은 편안한 분위기에서 커피를 즐기며 친구나 가족과 함께 시간을 보낼 수 있습니다. 매장마다 다양한 테마와 인테리어로 사람들에게 즐거움을 주고 있습니다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크를 통해 소통하는 앱이다. 사용자들은 팔로워들과 소통하며 일상을 공유하고 새로운 콘텐츠를 발견할 수 있다. 필터 기능을 통해 사진을 더욱 멋지게 꾸밀 수 있어 인기 있는 SNS 플랫폼 중 하나이다.',
 'ChatGPT는 자연어 처리 기술을 기반으로 한 대화형 인공지능 모델로, 사용자와 자연스럽게 대화하며 다양한 주제에 대한 정보나 도움을 제공합니다. 높은 정확도와 유연성을 통해 실제 대화와 거의 구분할 수 없는 수준의 대화를 제공하며, 다양한 분야에서 활용되고 있습니다. 사용자의 입력에 따라 학습하고 발전하는 ChatGPT는 계속해서 성능을 개선해 나가고 있습니다.',
 '한글은 한국어를 표기하는데 사용되는 문자로, 자음과 모음이 결합하여 음절을 이루는 특징을 가지고 있습니다. 한글은 조선 세종 대왕이 만든 문자로, 현대 한국어의 표기체계로 사용되고 있습니다. 한글은 세계에서 가장 과학적이고 효율적인 문자로 평가받고 있습니다.',
 '머신러닝은 컴퓨터 시스템이 데이터로부터 학습하여 패턴을 발견하고 예측하는 인공지능 기술입니다. 이를 통해 기존 데이터를 분석하고 새로운 데이터에 대한 결정을 내리는 데 사용됩니다. 머신러닝은 이미지 및 음성 인식, 자연어 처리, 금융 분석 등 다양한 분야에서 활용되고 있습니다.',
 '뉴런은 뇌와 신경계통의 기본 단위로서 전기적인 신호를 받아들이고 처리하여 다른 뉴런에 전달하는 세포입니다. 뉴런은 가지돌기, 축색돌기, 축색돌기 끝단으로 구성되어 있으며 시냅스를 통해 다른 뉴런과 연결되어 정보를 주고받습니다. 뉴런의 연결과 활성화에 따라 학습, 기억, 감정 등 다양한 뇌 기능이 조절됩니다.']

# async stream : astream

In [11]:
async for token in chain.astream({"topic":"YouTube"}):
    print(token, end="", flush=True)

YouTube는 동영상을 업로드하고 시청할 수 있는 온라인 동영상 플랫폼이다. 사용자들은 다양한 주제의 동영상을 시청하고 구독할 수 있으며, 댓글을 남기거나 좋아요를 누를 수도 있다. 또한 유명 인터넷 스타들이나 기업들도 YouTube를 통해 콘텐츠를 공유하고 홍보에 활용한다.

# async invoke : ainvoke

In [12]:
achain_process = chain.ainvoke({"topic":"NVIDIA"})

In [13]:
await achain_process

'NVIDIA는 그래픽 카드 및 시스템 온 칩 제조업체로, 주로 게임 및 전문가용 시각화 솔루션을 제공합니다. 또한 딥 러닝 및 인공지능 분야에서도 선도적인 기술을 제공하며, GPU 가속 컴퓨팅 기술을 선도하고 있습니다. 고성능 그래픽 처리 장치와 소프트웨어를 통해 혁신적인 시각화 및 연산 솔루션을 제공하여 전 세계적으로 널리 사용되고 있습니다.'

# async batch : abatch

In [14]:
abatch_process = chain.abatch(
    [
        {"topic":"meta"},
        {"topic":"Instagram"},
        {"topic":"tiktok"},
    ]
)

In [15]:
await abatch_process

['Meta는 다른 것을 설명하거나 분석하는 데 사용되는 개념이며, 주제나 대상을 넘어서 그 이면에 숨겨진 의미나 관계를 탐구하는 것을 의미합니다. 메타데이터, 메타언어 등과 같이 다양한 분야에서 사용되며, 주로 추상적이고 상위 수준의 개념을 나타냅니다. 메타적인 접근은 일반적으로 보다 깊은 이해와 인사이트를 제공하며, 복잡한 시스템이나 개념을 이해하는 데 도움이 됩니다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 사용자들은 팔로워들과 소통하고 일상을 공유할 수 있습니다. 해시태그를 통해 관심사에 맞는 콘텐츠를 찾을 수 있습니다.',
 'TikTok은 짧은 동영상을 공유하는 소셜미디어 플랫폼으로, 사용자들은 다양한 콘텐츠를 업로드하고 시청할 수 있습니다. 특히 음악, 춤, 코미디 등의 다양한 콘텐츠가 인기를 끌고 있습니다. 빠른 속도로 성장하고 있는 플랫폼으로, 트렌드를 만들고 새로운 스타를 발굴하는 역할을 하고 있습니다.']

# Parallel

In [16]:
from langchain_core.runnables import RunnableParallel

In [18]:
chain1 = (
    PromptTemplate.from_template("{country}의 언어는 무엇입니까?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

combine_chains = RunnableParallel(language=chain1, capital=chain2)

In [19]:
# chain2 실행
chain2.invoke({"country":"일본"})

'일본의 수도는 도쿄입니다.'

In [20]:
# 병렬 체인 실행 
combine_chains.invoke({"country":"대한민국"})

{'language': '대한민국의 언어는 한국어입니다.', 'capital': '대한민국의 수도는 서울입니다.'}

# Parallel + Batch

In [22]:
chain1.batch(
    [
        {"country":"대한민국"},
        {"country":"미국"},
    ]
)

['대한민국의 언어는 한국어입니다.',
 '미국의 공식 언어는 영어입니다. 하지만 미국 내에는 스페인어, 중국어, 한국어, 프랑스어 등 다양한 언어가 사용되고 있습니다.']

In [23]:
chain2.batch(
    [
        {"country":"대한민국"},
        {"country":"미국"},
    ]
)

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

In [24]:
combine_chains.batch(
    [
        {"country":"대한민국"},
        {"country":"미국"},
    ]
)

[{'language': '대한민국의 언어는 한국어입니다.', 'capital': '대한민국의 수도는 서울입니다.'},
 {'language': '미국의 주요 언어는 영어입니다. 그러나 스페인어, 중국어, 프랑스어, 독일어 등 다양한 언어가 미국 내에서 사용되고 있습니다.',
  'capital': '미국의 수도는 워싱턴 D.C.입니다.'}]

-----
* End of Document *